# LDA - Topic Modeling

In [12]:
import utils
import pandas as pd
import nltk

# nltk.download('brown')
# nltk.download('punkt')

from nltk.corpus import brown

In [28]:
conn = utils.create_connection('../scraping538.db')
query = "SELECT * FROM ARTICLES_538 LIMIT 100"
articles_all = pd.read_sql(query, con=conn)

articles_all.shape

(100, 14)

In [29]:
articles_all.head(2)

,title,DATE,hour,DATE_HOUR,author,filed_under,article_text,mini_bio,url,project,spider,server,DATE_IMPORT,PK_ID
0,Can The Warriors Still Win Without Curry? And ...,2019-10-31 00:00:00,2:08 PM,2019-10-31 14:08:00,Chris Herring,NBA,Just when you thought things couldn’t get any ...,Chris Herring is a senior sportswriter for Fiv...,https://fivethirtyeight.com/features/can-the-w...,scrapping_538,all_articles,mrmoe,2019-11-02 21:54:30.067303,31-10-2019_Can The Warriors Still Win Without ...
1,The Impeachment Inquiry Will Now Be Televised,2019-10-31 00:00:00,1:11 PM,2019-10-31 13:11:00,Clare Malone,The Spin Cycle,"Welcome to The Spin Cycle, a semi-regular look...",Clare Malone is a senior political writer for ...,https://fivethirtyeight.com/features/the-impea...,scrapping_538,all_articles,mrmoe,2019-11-02 21:54:30.076293,31-10-2019_The Impeachment Inquiry Will Now Be...


In [30]:
data = []
 
for article_id in articles_all.index:
#     document = ' '.join(brown.words(fileid))
    document = articles_all['article_text'][article_id]
    data.append(document)
 
NO_DOCUMENTS = len(data)
print(NO_DOCUMENTS)
print(data[:1])

100
['Just when you thought things couldn’t get any worse for the Golden State Warriors — who had already been  on the wrong end of two blowouts  in their first week of play and were headed for a third on Wednesday — worse came crashing down onto the floor of the new Chase Center. \n In the third quarter, Stephen Curry dribbled to his left in transition, preparing to go in for a layup, when Phoenix Suns center Aron Baynes slid in front of the airborne point guard, causing him to tumble. Baynes then fell down, too, with his weight coming down on Curry’s left hand. The superstar couldn’t move his hand and walked to the locker room, and shortly after running some tests, the Warriors  put out the news : Curry’s left hand was broken. He will be out for the foreseeable future. \n \n On the most basic level, the injury is devastating for a struggling team that, until very recently,  had it better than any other team  these past few seasons — largely because of Curry’s impact. But even now, in

In [31]:
import re
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
 
NUM_TOPICS = 10
STOPWORDS = stopwords.words('english')
 
def clean_text(text):
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
    return cleaned_text
 
# For gensim we need to tokenize the data and filter out stopwords
tokenized_data = []
for text in data:
    tokenized_data.append(clean_text(text))
 
 
# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
 
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
 
# Have a look at how the 20th document looks like: [(word_id, count), ...]
# print(corpus[20])
# [(12, 3), (14, 1), (21, 1), (25, 5), (30, 2), (31, 5), (33, 1), (42, 1), (43, 2),  ...
 
# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

In [32]:
print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)

LDA Model:
Topic #0: 0.007*"percent" + 0.005*"would" + 0.005*"week" + 0.004*"warren" + 0.004*"two" + 0.004*"new" + 0.004*"season" + 0.003*"team" + 0.003*"think" + 0.003*"one"
Topic #1: 0.010*"percent" + 0.004*"raptor" + 0.004*"would" + 0.004*"voters" + 0.004*"season" + 0.004*"also" + 0.004*"think" + 0.004*"warren" + 0.004*"one" + 0.003*"team"
Topic #2: 0.014*"percent" + 0.005*"raptor" + 0.004*"warren" + 0.004*"polls" + 0.004*"also" + 0.004*"season" + 0.004*"biden" + 0.004*"one" + 0.003*"points" + 0.003*"player"
Topic #3: 0.009*"percent" + 0.007*"season" + 0.005*"would" + 0.004*"like" + 0.004*"one" + 0.004*"team" + 0.004*"could" + 0.004*"biden" + 0.004*"also" + 0.004*"warren"
Topic #4: 0.008*"percent" + 0.006*"season" + 0.005*"teams" + 0.005*"team" + 0.005*"like" + 0.004*"raptor" + 0.004*"game" + 0.004*"one" + 0.004*"points" + 0.004*"two"
Topic #5: 0.007*"warren" + 0.005*"democrats" + 0.005*"week" + 0.005*"percent" + 0.005*"biden" + 0.004*"trump" + 0.004*"think" + 0.004*"voters" + 0.004